In [4]:
from collections import Counter
from functools import reduce

from transformers import AutoTokenizer
from nltk.tokenize import word_tokenize
import seaborn as sns
import pandas as pd

from src.preprocessing import load_semeval_taskb

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pierr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
TOKEN = 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf', token=TOKEN)

In [ ]:
full, train, test = load_semeval_taskb()

Labels: Labels: 1 (ironic by clash), 2 (situational irony), 3 (other irony), 0 (not ironic)

# Label Distribution

In [ ]:
label_distrib = pd.DataFrame(
    [(x[0][0], x[0][1], x[1], round(x[1] / (full['split'] == x[0][0]).sum(), 3)) for x in full.groupby(['split', "label_id"])['label_id'].count().items()],
    columns=['split', 'label_id', 'count', 'ratio']
)

sns.set(rc={'figure.figsize':(16, 6)})
ax = sns.barplot(
    data=label_distrib,
    x='label_id',
    y='count',
    hue='split',
    hue_order=['train', 'test']
)
ax.bar_label(
    ax.containers[0], 
    labels=[f'{c:.0f} ({r})' for c, r in label_distrib[label_distrib.split == 'train'][['count', 'ratio']].values]
) 
ax.bar_label(
    ax.containers[1],     
    labels=[f'{c:.0f} ({r})' for c, r in label_distrib[label_distrib.split == 'test'][['count', 'ratio']].values]
) 

## Llama2 Tokenized text count distribution

In [ ]:
full['llama2_tokens'] = full.text.apply(lambda x: [tokenizer.decode(tid) for tid in tokenizer.encode(x)[1:]])
full['llama2_tokens_len'] = full.llama2_tokens.apply(lambda x: len(x))

In [ ]:
sns.boxplot(full, x='label_id', y='llama2_tokens_len', hue='split')

In [ ]:
# Dropped value count for token_len < 100
full.groupby(['split', "label_id"])['label_id'].count() - full[full['llama2_tokens_len'] < 100].groupby(['split', "label_id"])['label_id'].count()

### Drop > 100 tokens

In [ ]:
full = full[full['llama2_tokens_len'] < 100].reindex()
full.info()

## Word distribution

In [ ]:
full['nltk_tokens'] = full.text.apply(lambda x: [t.strip().lower() for t in word_tokenize(x)])

In [ ]:
import string


en_stopwords = stopwords.words('english') + list(string.punctuation) + [
    '...', "''", '``', 'http', "'s", "n't"
]
def get_counter(texts, stopwords=en_stopwords):
    tokens = reduce(lambda a,b: a+b, texts, [])
    tokens = list(filter(lambda x: x not in stopwords, tokens))
    return Counter(tokens)

full_tokens = get_counter(full.nltk_tokens.values)

label_tokens = {}
for label_id in sorted(full.label_id.unique()):
    label_tokens[label_id] = get_counter(full[full.label_id == label_id].nltk_tokens.values)

In [ ]:
print("##### FULL #####")
print(full_tokens.most_common(20))
for label_id in sorted(full.label_id.unique()):
    print(f"##### {label_id} #####")
    print(label_tokens[label_id].most_common(20))